# 英文问答

In [ ]:
def  is_variable (pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])

In [85]:
def pat_match(pattern, saying):
    if not pattern or not saying: return []
    
    if is_variable(pattern[0]):
        return [(pattern[0], saying[0])] + pat_match(pattern[1:], saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match(pattern[1:], saying[1:])

In [89]:
pat_match("?X greater than ?Y and ?Z".split(), "3 greater than 2 and 1".split())

[('?X', '3'), ('?Y', '2'), ('?Z', '1')]

In [110]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [111]:
def subsitite(rule, parsed_rules):
    if not rule: return []
    
    return [parsed_rules.get(rule[0], rule[0])] + subsitite(rule[1:], parsed_rules)

In [112]:
subsitite("What if you mean if you got a ?X".split(),pat_to_dict([('?X', 'iPhone')]))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [113]:
got_patterns = pat_match("I want ?X".split(), "I want iPhone".split())
got_patterns

[('?X', 'iPhone')]

In [114]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [115]:
john_pat = pat_match('?P needs ?X'.split(), "John needs vacation".split())
john_pat

[('?P', 'John'), ('?X', 'vacation')]

In [116]:
subsitite ("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'vacation', '?']

In [117]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need vacation ?'

In [118]:
defined_patterns = {
    "I need ?X": ["Image you will get ?X soon", "Why do you need ?X ?"], 
    "My ?X told me something": ["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [119]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])

In [120]:
is_pattern_segment('?*P')

True

In [121]:
def is_match(rest, saying):
    if not rest and not saying:
        return True
    if not all(a.isalpha() for a in rest[0]): 
        return True
    if rest[0] != saying[0]: 
        return False
    return is_match(rest[1:], saying[1:])


def segment_match(pattern, saying):
    if pattern[0] == saying[0]:
        return segment_match(pattern[1:],saying[1:])
    elif is_pattern_segment(pattern[0]):
        seg_pat, rest = pattern[0], pattern[1:]
        seg_pat = seg_pat.replace('?*', '?')

        if not rest: return (seg_pat, saying), len(saying)    
    
        for i, token in enumerate(saying):
            if rest[0] == token and is_match(rest[1:], saying[(i+1):]):  
                return (seg_pat, saying[:i]), i
    
        return [],len(saying)
    else:
        return []

def  segment_match2(pattern, saying):
    if pattern[0] == saying[0]:
        return segment_match(pattern[1:],saying[1:])
    elif is_pattern_segment(pattern[0]):
        seg_pat, rest = pattern[0], pattern[1:]
        seg_pat = seg_pat.replace('?*', '?')
        if not rest: 
            return (seg_pat, saying), len(saying)    

        for i, token in enumerate(saying):
            if rest[0] == token:  
                return (seg_pat, saying[:i]), i

        return [],len(saying)  #(seg_pat, saying), len(saying)
    else:
        return []

In [122]:
segment_match('?*X hello ?*Y'.split(), "I was a pig".split())

([], 4)

In [123]:

fail = [True, None]

def pat_match_with_seg(pattern, saying):
    if not pattern or not saying: return []
    
    pat = pattern[0]
    
    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        if match:
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
        else:
            return []
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return []

In [124]:
pat_match_with_seg('I was ?*X'.split(), "I was Mike".split())

[('?X', ['Mike'])]

In [125]:
def pat_to_dict(patterns):
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in patterns}

In [126]:
' '.join(subsitite("Why do you neeed ?X ?".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split()))))

'Why do you neeed an iPhone ?'

In [162]:
def get_response(saying, rules):
    k = 0
    pattern = []
    for key in rules:
        pattern = pat_match_with_seg(key.split(),saying.split())
        print(key.split(),saying.split())
        print(pattern)
        if pattern: 
            rules = rules.get(key)[0].split()
            return ' '.join(subsitite(rules, pat_to_dict(pattern)))
    return 'No response'
         

In [163]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."]
}

In [164]:
get_response("I was a pig ", rules)

['I', 'was', '?*X'] ['I', 'was', 'a', 'pig']
[('?X', ['a', 'pig'])]


'Were you really a pig ?'

In [130]:
rules2 = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?']
}

In [131]:
get_response('mama I was very happy', rules2)

'Were you really'

# 中文问答

In [132]:
import jieba
import re

In [257]:
def get_response_Chinese(saying, rules):
    k = 0
    pattern = []
    for key in rules:
        pattern = pat_match_with_seg(get_pattern(key),','.join(jieba.cut(saying)).split(','))
        print(get_pattern(key), ','.join(jieba.cut(saying)).split(','))
        print(pattern)
        if pattern: 
            rules = get_pattern(rules.get(key)[0])
            print(rules)
            return ' '.join(subsitite(rules, pat_to_dict(pattern)))
    return 'No response'

In [258]:
def get_pattern(text):
    text = text.replace('?*x','xxx').replace('?*y','yyy').replace('?*z','yyz').replace('?x','xx').replace('?y','yy')
    seg = ','.join(jieba.cut(text))
    seg = seg.replace('xxx','?*x').replace('yyy','?*y').replace('?z','zz').replace('xx','?x').replace('yy','?y')
    return seg.split(',')


In [259]:
get_pattern('?*x我喜欢你?*y')

['?*x', '我', '喜欢', '你', '?*y']

In [260]:
rules3 = {
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x每个人?*y': ['我确定不是人人都是?x', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人']
}

In [261]:
saying = '还好每个人'

In [262]:
get_response_Chinese(saying, rules3)

['?*x', '妈妈', '?*y'] ['还好', '每个', '人']
[]
['?*x', '我', '想', '?*y'] ['还好', '每个', '人']
[]
['?*x', '喜欢', '?*y'] ['还好', '每个', '人']
[]
['?*x', '你好', '?*y'] ['还好', '每个', '人']
[]
['?*x', '如果', '?*y'] ['还好', '每个', '人']
[]
['?*x', '我', '记得', '?*y'] ['还好', '每个', '人']
[]
['?*x', '讨厌', '?*y'] ['还好', '每个', '人']
[]
['?*xAI?*y'] ['还好', '每个', '人']
[]
['?*x', '对不起', '?*y'] ['还好', '每个', '人']
[]
['?*x', '我', '?yz', '梦见', '?*y'] ['还好', '每个', '人']
[]
['?*x', '我', '想要', '?*y'] ['还好', '每个', '人']
[]
['?*x', '爸爸', '?*y'] ['还好', '每个', '人']
[]
['?*x', '每个', '人', '?*y'] ['还好', '每个', '人']
[('?x', ['还好'])]
['我', '确定', '不是', '人人', '都', '是', '?x']


'我 确定 不是 人人 都 是 还好'

# 关于数据驱动和人工智能的想法

<font color=#0099ff size=3 face="黑体">
    
    这个程序可以完成符合特定模式的基本问答，但还是很僵化。如果问句格式与设置不符，或模式里不包含问句内容，就不能得到回答。
    
    我觉得可以改进匹配模式，现有的匹配模式必须使用户问句和设定问句有完全一致的词语，可以改成意思相近即可匹配。
    
    我认为数据驱动是通过收集并分析数据形成决策，区别于rule based（人为的预先制定决策），这样做的好处是随着外界环境的变化，数据产生变化，形成的决策也会随之改变，而通过人来制定决策就很僵硬，一旦情况发生变化就需要手动改变决策，所以从这个角度将，数据驱动的决策生成是自动完成的，具有“智能性”。
